Following the same steps to find the best directors (avg note of their movies in the last 2 decades), follow the same schema to find the writers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pymysql
import getpass

%matplotlib inline

In [2]:
# I connect with a db locally
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="root",
                       passwd=getpass.getpass(),
                       db="movies")

········


In [3]:
q = """
SELECT
    tp.nconst,                     
    count(*) as count,
    AVG(averageRating) as rating,
    SUM(numVotes) as votes
FROM
    title_principals tp
JOIN
    title_basics tb
    ON tp.tconst = tb.tconst
JOIN
    title_ratings tr
    ON tp.tconst = tr.tconst
WHERE
    tp.category = "writer"
    AND tb.titleType = "movie"
    AND tb.startYear BETWEEN 2000 AND 2019
    AND tb.runtimeMinutes > 40
    AND isAdult = 0
GROUP BY
    tp.nconst
HAVING
    SUM(numVotes) > 1000000
ORDER BY
    3
DESC
"""
# nconst is the unique identifier that imdb uses for identify people
# tconst is the unique identifier that imdb uses for identify movies
# we consider only movies between 2000 and 2019
# in this query we search for directors
# Minimum popularity required: threshold of at least 1.000.000 votes (in the sum of the whole movies in the timeframe)
# Consider minimum lenght of 80 minutes (according current standards and AFI)

In [4]:
writers = pd.read_sql(q,conn)


In [5]:
writers.describe().T

,count,mean,std,min,25%,50%,75%,max
count,186.0,6.983871e+00,4.096770e+00,1.0,5.000000e+00,6.000,9.00,33.0
rating,186.0,6.751645e+00,5.753358e-01,5.2,6.331591e+00,6.715,7.08,8.7
votes,186.0,1.953157e+06,1.505295e+06,1007284.0,1.168336e+06,1408815.500,1986916.50,12131181.0


In [6]:
writers.head(20)

,nconst,count,rating,votes
0,nm0801728,1,8.700000,1408790.0
1,nm0538320,1,8.500000,1122466.0
2,nm1793993,1,8.500000,1113258.0
3,nm0634300,6,8.366667,7231224.0
4,nm0067789,1,8.200000,1092028.0
5,nm0714114,4,7.800000,1825280.0
6,nm2441763,3,7.800000,1012655.0
7,nm5037683,2,7.800000,1522862.0
8,nm0909638,9,7.711111,6927702.0
9,nm0101991,9,7.711111,6927702.0


the most of writers with "best avg rating has writen only 1 movie"

we set a treshold of 3 movies

In [7]:
writers.sort_values(by="count", ascending=False)

,nconst,count,rating,votes
170,nm0000108,33,6.051515,2693333.0
76,nm0498278,26,6.834615,12131181.0
166,nm0000175,24,6.066667,1999068.0
68,nm0456158,17,6.917647,8487138.0
131,nm0159039,16,6.443750,1285161.0
...,...,...,...,...
16,nm4160687,2,7.450000,1226373.0
4,nm0067789,1,8.200000,1092028.0
1,nm0538320,1,8.500000,1122466.0
2,nm1793993,1,8.500000,1113258.0


to consider only writers with, at least, 3 movies written in the last 20 years

In [8]:
writers[writers["count"].ge(3)].head(20)

,nconst,count,rating,votes
3,nm0634300,6,8.366667,7231224.0
5,nm0714114,4,7.800000,1825280.0
6,nm2441763,3,7.800000,1012655.0
8,nm0909638,9,7.711111,6927702.0
9,nm0101991,9,7.711111,6927702.0
10,nm0866058,8,7.587500,6311857.0
11,nm0460141,9,7.577778,4340928.0
12,nm1578335,7,7.571429,3616732.0
13,nm0565026,5,7.560000,1020098.0
14,nm0798788,4,7.550000,1316881.0


In [9]:
# for retrieve the names, we will webscrap
import requests
import bs4

# define function to scrap name from imdb
def get_name_title(name_code):
    # define url
    base = "https://www.imdb.com/name/"
    url = base + name_code
    
    # get the soup
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, "html.parser")
    
    # get name
    name = soup.title.contents[0]
    
    return name[:-7]

In [10]:
# test
get_name_title("nm0634300")

'Jonathan Nolan'

In [12]:
nconst_list = [n for n in writers.nconst]
get_name_title(nconst_list[0])

'Stephen Sinclair'

In [13]:
list_of_names = [get_name_title(n) for n in nconst_list]

In [14]:
list_of_names

['Stephen Sinclair',
 'Alan Mak',
 'Christopher Priest',
 'Jonathan Nolan',
 'Jordan Belfort',
 'Jim Reardon',
 'Cressida Cowell',
 'Andy Lanning',
 'Fran Walsh',
 'Philippa Boyens',
 'J.R.R. Tolkien',
 'Steve Kloves',
 'Michael Arndt',
 'Anthony McCarten',
 'Scott Silver',
 'J.K. Rowling',
 'Jim Starlin',
 'Joe Simon',
 'Dennis Lehane',
 'David Franzoni',
 'Peter Morgan',
 'Lawrence Kasdan',
 'Robert Ludlum',
 'Aaron Sorkin',
 'Jay Wolpert',
 'Bob Peterson',
 'Charlie Kaufman',
 'Paul Haggis',
 'Larry Lieber',
 'William Nicholson',
 'Dave Gibbons',
 'Drew Goddard',
 'Rick Jaffa',
 'Amanda Silver',
 'Meg LeFauve',
 'Erik Sommers',
 'Steven Zaillian',
 'William Steig',
 'Nicole Perlman',
 'Ian Fleming',
 'Jane Goldman',
 'Christopher Markus',
 'Stephen McFeely',
 'Bruce Geller',
 'Don Heck',
 'Simon Beaufoy',
 'William Monahan',
 'John Logan',
 'Pierre Boulle',
 'Dan Abnett',
 'Eric Roth',
 'Mark Millar',
 'Chris McKenna',
 'Drew Pearce',
 'Scott Frank',
 'Terry Rossio',
 'Charles Leavi

according to imdb, are included writers of the original source (JK Rowlings, JRR Tolkien), mixed with actual screenwriters

In [15]:
# add names of writers
writers = writers.assign(name=list_of_names)

In [18]:
# round numbers
writers.rating = writers.rating.round(2)
writers.votes = writers.votes.astype(int)

In [19]:
writers.head(20)

,nconst,count,rating,votes,name
0,nm0801728,1,8.70,1408790,Stephen Sinclair
1,nm0538320,1,8.50,1122466,Alan Mak
2,nm1793993,1,8.50,1113258,Christopher Priest
3,nm0634300,6,8.37,7231224,Jonathan Nolan
4,nm0067789,1,8.20,1092028,Jordan Belfort
5,nm0714114,4,7.80,1825280,Jim Reardon
6,nm2441763,3,7.80,1012655,Cressida Cowell
7,nm5037683,2,7.80,1522862,Andy Lanning
8,nm0909638,9,7.71,6927702,Fran Walsh
9,nm0101991,9,7.71,6927702,Philippa Boyens


In [20]:
# writers.to_csv("../data/top_directors_imdb.csv", index=False)